In [ ]:
import fsspec
import xarray as xr
import hvplot.xarray
import hvplot.pandas
import hvplot.dask
# import intake
import os
import warnings
# import intake_xarray
import geopandas as gpd
from pynhd import NLDI, WaterData
# from dask.distributed import LocalCluster, Client
# from gdptools.helpers import get_shp_bounds_w_buffer, build_subset
# from helpers import configure_cluster

warnings.filterwarnings('ignore')

In [ ]:
# USGS gage 01482100 Delaware River at Del Mem Bridge at Wilmington De
gage_id = '01482100'
nldi = NLDI()
del_basins = nldi.get_basins(gage_id)
huc12_basins = WaterData('huc12').bygeom(del_basins.geometry[0])

import pandas as pd
param_json = "https://mikejohnson51.github.io/opendap.catalog/cat_params.json"
grid_json = "https://mikejohnson51.github.io/opendap.catalog/cat_grids.json"
params = pd.read_json(param_json)
grids = pd.read_json(grid_json)

_id = "gridmet"
_varname = "daily_maximum_temperature"
tc = params.query("id == @_id & varname == @_varname")
print(type(tc), len(tc))
tc

In [ ]:
# Create a dictionary of parameter dataframes for each variable
vars = ["tmmn", "tmmx"]
var_params = []
for var in vars:
    var_params.append(params.query("id == @_id & variable == @var"))
param_dict = dict(zip(vars, var_params))
param_dict.get('tmmn')

In [ ]:
# Create a dictionary of grid dataframes for each variable
var_grid = []
for var in vars:
    gridid = param_dict.get(var)['grid_id'].values[0]
    var_grid.append(grids.query('grid_id == @gridid'))
grid_dict = dict(zip(vars, var_grid))
grid_dict.get('tmmn')

In [ ]:
import tempfile
from gdptools.helpers import calc_weights_catalog
wghtf = calc_weights_catalog(
    params_json=param_dict.get("tmmn"),
    grid_json=grid_dict.get("tmmn"),
    shp_file=huc12_basins,
    shp_poly_idx='huc12',
    wght_gen_file=tempfile.NamedTemporaryFile().name,
    wght_gen_proj=6931,
    )

tmp = 0

In [ ]:
from gdptools.run_weights_engine import RunWghtEngine
eng = RunWghtEngine()
eng.initialize(
    param_dict=param_dict,
    grid_dict=grid_dict,
    wghts=wghtf,
    gdf=huc12_basins,
    gdf_poly_idx='huc12',
    start_date="2020-01-01",
    end_date="2020-01-07"
)
ngdf, nvals = eng.run(numdiv=1)    